# 02. 웹 페이지 스크레이핑하기

## 라이브러리 설치하기

In [1]:
import requests

requests.__version__

'2.26.0'

In [2]:
import lxml

lxml.__version__

'4.6.3'

In [3]:
import cssselect

cssselect.__version__

'1.1.0'

## 웹에 있는 리소스 추출하기
- 단순하게 웹 페이지를 추출하는 것이라면 urllib만으로도 충분하지만 HTTP 헤더에 정보를 넣어 접근하거나 BASIC 인증을 하려면 힘듬

### Requests를 사용해서 웹에 있는 리소스 추출하기
- HTTP 메서드와 같은 이름의 메서드를 제공

```python
import requests

requests.get('URL')
requests.post('URL')
requests.put('URL')
requests.delete('URL')
requests.head('URL') # header 추출하기

# GET 메서드에 매개 변수 추가
requests.get('URL', params={'key1':'value1', 'key2':'value2'})
```

- API를 호출할 때 대부분 POST 요청의 body에 JSON 인코딩한 매개 변수를 넣는 경우가 많음

```python
import json
import requests

requests.post('URL', data=json.dumps({'key1':'value1', 'key2':'value2'}))
```

### Requests의 응답
- GET 등의 메서드의 반환값은 Response 클래스이며 status_code 변수에 접근하면 상태 코드를 확인할 수 있음

In [7]:
import requests

r = requests.get('https://jsonplaceholder.typicode.com/users/1')
r.status_code

200

- 응답 본문(body)가 byte 자료형일 경우 content, str 자료형일 경우 texttest에 접근

In [8]:
r.text

'{\n  "id": 1,\n  "name": "Leanne Graham",\n  "username": "Bret",\n  "email": "Sincere@april.biz",\n  "address": {\n    "street": "Kulas Light",\n    "suite": "Apt. 556",\n    "city": "Gwenborough",\n    "zipcode": "92998-3874",\n    "geo": {\n      "lat": "-37.3159",\n      "lng": "81.1496"\n    }\n  },\n  "phone": "1-770-736-8031 x56442",\n  "website": "hildegard.org",\n  "company": {\n    "name": "Romaguera-Crona",\n    "catchPhrase": "Multi-layered client-server neural-net",\n    "bs": "harness real-time e-markets"\n  }\n}'

### JSON 파싱하기
- Response 클래스의 json 메서드 : body가 JSON 형식이라면 파싱하여 딕셔너리 자료형으로 만들어주는 메서드

In [9]:
json = r.json()

In [10]:
json

{'id': 1,
 'name': 'Leanne Graham',
 'username': 'Bret',
 'email': 'Sincere@april.biz',
 'address': {'street': 'Kulas Light',
  'suite': 'Apt. 556',
  'city': 'Gwenborough',
  'zipcode': '92998-3874',
  'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
 'phone': '1-770-736-8031 x56442',
 'website': 'hildegard.org',
 'company': {'name': 'Romaguera-Crona',
  'catchPhrase': 'Multi-layered client-server neural-net',
  'bs': 'harness real-time e-markets'}}

## XPath와 CSS 선택자
- XPath : XML Path Language의 줄임말로 XML로 작성된 문서의 특정 부분을 선택할 때 사용하는 언어

### XPath로 요소 지정하기

```html
<html>
    <body>
        <h1>도서</h1>
        <div class="book">
            <p class="title">파이썬을 활용한 크롤러 개발과 스크레이핑 입문</p>
            <a href="/python-crawler"></a>
        </div>
    </body>
</html>
```
- h1 요소의 위치 : /html/body/h1
- //로 시작하는 경우는 전체 요소를 대상으로 함
    - //div : HTML 소스 내부의 모든 div를 지정

### CSS 선택자로 요소 지정하기
- h1 요소의 위치 : html > body > h1
- 클래스 이름을 지정 : `.<클래스 이름>`
- id 이름을 지정 : `#<id 이름>`

## HTML 소스 분석하기
- https://wikibook.co.kr/python-crawler/ 페이지에서 도서 제목 추출하기

### 추출하고 싶은 요소의 XPath와 CSS 선택자 추출하기
- 검사 - 요소 선택 - 마우스 우클릭 - Copy - Copy selector (CSS) / Copy XPath (XPath)
- XPath : `//*[@id="content"]/div[1]/div[2]/h1`
- CSS : `#content > div:nth-child(1) > div.col-md-9 > h1`

### lxml로 스크레이핑하기

In [11]:
import requests
import lxml.html

r = requests.get('https://wikibook.co.kr/python-crawler/')
html = r.text

In [12]:
# lxml을 사용해 HtmlElement 클래스 객체로 변환
# 매개 변수에 HTML 소스 코드를 지정해서 fromstring 메서드 호출
root = lxml.html.fromstring(html)

In [15]:
# HtmlElement의 xpath 메서드에 XPath를 지정해서 요소 추출
# 여러 개인 경우도 있기 때문에 항상 리스트로 반환
titleElement = root.xpath('//*[@id="content"]/div[1]/div[2]/h1')

In [16]:
titleElement

[<Element h1 at 0x1bef082fef8>]

In [17]:
# 텍스트 추출
titleElement[0].text

'파이썬을 활용한 크롤러 개발과 스크레이핑 입문'

In [18]:
# 태그 이름, 태그 속성 등도 추출할 수 있음
titleElement[0].tag

'h1'

In [19]:
titleElement[0].attrib

{'class': 'main-title'}

In [31]:
# CSS 선택자로 추출할 때 HtmlElement 클래스의 cssselect 메서드 사용
for i in range(1, 5):
    linkA = root.cssselect('#book-stores > li:nth-child({}) > a'.format(i))
    print(linkA[0].attrib['href'])

http://www.yes24.com/Product/Goods/76488159
http://www.kyobobook.co.kr/product/detailViewKor.laf?barcode=9791158391645
http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.prdNo=312152476
http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=198765108


In [33]:
linkAs = root.cssselect('#book-stores > li > a')

In [34]:
for linkA in linkAs:
    print(linkA.attrib['href'])

http://www.yes24.com/Product/Goods/76488159
http://www.kyobobook.co.kr/product/detailViewKor.laf?barcode=9791158391645
http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.prdNo=312152476
http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=198765108


In [38]:
linkAs = root.cssselect('#book-stores > li > a > img')

In [41]:
for linkA in linkAs:
    print(linkA.attrib['src'])

https://wikibook.co.kr/images/img/yes24.png
https://wikibook.co.kr/images/img/kyobo.png
https://wikibook.co.kr/images/img/interpark.png
https://wikibook.co.kr/images/img/aladin.png
